# dataset classification
3/15/24

The point of this document is to test the dataset class for classification and see if our changes work properly, we will also do various tests with the model to ensure it's working

In [1]:
print('test')

test


In [2]:
#first let's load in the dataset class
from src.dataloaders.datasets.DNase_dataset import DNaseDataset
from src.dataloaders.datasets.hg38_char_tokenizer import CharacterTokenizer
tokenizer = CharacterTokenizer( #make sure to fix the tokenizer too
                characters=['A', 'C', 'G', 'T', 'N', 'S', 'U', 'V', 'W', 'X', 'Y', 'Z'],
                model_max_length=1024 + 2,  # add 2 since default adds eos/eos tokens, crop later
                add_special_tokens=False,
                padding_side='left'
            )

dataset = DNaseDataset(max_length = 1024, split = 'test', tokenizer=tokenizer, rc_aug = False, tokenizer_name='char', add_eos='True', filter = True)


In [3]:
a,b = dataset[0]

In [4]:
print(a)
print(b) #this is the normal one, now set classificaiton to true

tensor([12, 13, 14,  ...,  9,  9, 10])
tensor([-10.])


In [5]:
classification = DNaseDataset(max_length = 1024, split = 'test', tokenizer=tokenizer, rc_aug = False, tokenizer_name='char', add_eos='True', filter = True, classification = True)
a,b = dataset[0]
print(a)
print(b)

tensor([12, 13, 14,  ...,  9,  9, 10])
tensor([-10.])


In [6]:
a2,b2 = classification[0]
print(a2)
print(b2) #this is the classification one, and we see that it is indeed the two outputs in a tuple!!

tensor([12, 13, 14,  ...,  9,  9, 10])
(tensor([0]), tensor([-10.]))


In [8]:
import torch
torch.allclose(a,a2)

True

In [10]:
#let's find several other examples
reg_list = []
class_list = []
for i in range(10):
    a,b = dataset[i]
    a2,b2 = classification[i]
    print(torch.allclose(a,a2))
    # print(b)
    # print(b2)
    # print('')
    reg_list.append(b)
    class_list.append(b2)
print(reg_list)
print(class_list)

True
True
True
True
True
True
True
True
True
True
[tensor([-10.]), tensor([-0.4581]), tensor([-10.]), tensor([-10.]), tensor([-1.8792]), tensor([-10.]), tensor([-2.3478]), tensor([-1.8173]), tensor([-0.6197]), tensor([-3.1970])]
[(tensor([0]), tensor([-10.])), (tensor([1]), tensor([-0.4581])), (tensor([0]), tensor([-10.])), (tensor([0]), tensor([-10.])), (tensor([1]), tensor([-1.8792])), (tensor([0]), tensor([-10.])), (tensor([1]), tensor([-2.3478])), (tensor([1]), tensor([-1.8173])), (tensor([1]), tensor([-0.6197])), (tensor([1]), tensor([-3.1970]))]


In [11]:
#seems to work just fine, now let's test the multitasking model and the ctst model
from src.dataloaders.datasets.DNase_allcelltypes import DNaseAllCellTypeDataset
classification_multitask = DNaseAllCellTypeDataset(max_length = 1024, split = 'test', tokenizer=tokenizer, rc_aug = False, tokenizer_name='char', add_eos='True', filter = True, classification = True)
a,b = classification_multitask[0]
print(a)
print(b)

tensor([8, 7, 9,  ..., 9, 7, 8])
(tensor([0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1,
        0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1,
        0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0,
        0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1,
        0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1,
        0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0]), tensor([-1.0000e+01, -4.5808e-01, -1.0000e+01, -1.0000e+01, -1.8792e+00,
        -1.0000e+01, -2.3478e+00, -1.8173e+00, -6.1973e-01, -3.1970e+00,
        -3.1362e+00, -6.3732e-01, -1.0000e+01, -1.0000e+01, -1.0000e+01,
        -1.0000e+01, -1.0000e+01, -1.0000e+01, -1.0000e+01, -1.6359e+00,
        -2.9812e+00, -1.0000e+01, -1.0000e+01, -1.9674e+00, -1.0000e+01,
        -1.0000e+01, -1.0000e+01, -1.1343e+00, -1.4971e+00, 

In [12]:
b[0] #tensor of all the classes, the associated regression values are there

tensor([0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1,
        0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1,
        0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0,
        0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1,
        0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1,
        0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0])

In [14]:
print(a)
print(a2)
torch.allclose(a[3:-4],a2[7:]) #this makes sense, as  this is what we use with our ism utils to align the sequences

tensor([8, 7, 9,  ..., 9, 7, 8])
tensor([12, 13, 14,  ...,  9,  9, 10])


True

In [18]:
#now let's loop and make sure the a matches and the b matches
for i in range(161):
    tempa,tempb = classification[i]
    # print(torch.allclose(a,tempa))
    if not torch.allclose(b[0][i],tempb[0]):
        print('no')
    if not torch.allclose(b[1][i],tempb[1]):
        print('no')
    # print(torch.allclose(b[0][i],b2[0]))
    # print(torch.allclose(b[1][i],b2[i]))
    if i<10:
        print(b[0][i])
        print(b[1][i])
        print(tempb)
        print('')
    # reg_list.append(b)
    # class_list.append

tensor(0)
tensor(-10.)
(tensor([0]), tensor([-10.]))

tensor(1)
tensor(-0.4581)
(tensor([1]), tensor([-0.4581]))

tensor(0)
tensor(-10.)
(tensor([0]), tensor([-10.]))

tensor(0)
tensor(-10.)
(tensor([0]), tensor([-10.]))

tensor(1)
tensor(-1.8792)
(tensor([1]), tensor([-1.8792]))

tensor(0)
tensor(-10.)
(tensor([0]), tensor([-10.]))

tensor(1)
tensor(-2.3478)
(tensor([1]), tensor([-2.3478]))

tensor(1)
tensor(-1.8173)
(tensor([1]), tensor([-1.8173]))

tensor(1)
tensor(-0.6197)
(tensor([1]), tensor([-0.6197]))

tensor(1)
tensor(-3.1970)
(tensor([1]), tensor([-3.1970]))



In [20]:
#let's jsut test it with a very random ccre number 83728
a,b = classification_multitask[83728]
print(a)
print(b[0][0],b[1][0])
a2,b2 = classification[83728*161]
print(a2)
print(b2)

tensor([ 8, 10,  8,  ..., 10,  9,  7])
tensor(1) tensor(0.5242)
tensor([12, 13, 14,  ...,  8,  7,  8])
(tensor([1]), tensor([0.5242]))


In [22]:
print(torch.allclose(a[3:-4],a2[7:]))
print(a[3:-4])
print(a2[7:])

True
tensor([7, 9, 7,  ..., 8, 7, 8])
tensor([7, 9, 7,  ..., 8, 7, 8])


In [23]:
#finally let's test the ctst model
from src.dataloaders.datasets.DNase_ctst_dataset import DNaseCtstDataset
classification_ctst = DNaseCtstDataset(max_length = 1024, split = 'test', tokenizer=tokenizer, rc_aug = False, tokenizer_name='char', add_eos='True', filter = True, classification = True)
a,b = classification_ctst[0]
print(a)
print(b)
a2,b2 = classification[0]
print(a2)
print(b2)

tensor([19,  8,  7,  ...,  9,  7,  8])
(tensor([0]), tensor([-10.]))
tensor([12, 13, 14,  ...,  9,  9, 10])
(tensor([0]), tensor([-10.]))


In [24]:
print(len(classification_ctst))
print(len(classification))

16945572
16945572


In [28]:
#and now do a for loop that just randomly generates indices between 0 and the length of the dataset
for i in range(1000): #2 seconds for 1000... and all match
    idx = torch.randint(0,16945572,(1,))
    a,b = classification_ctst[idx]
    a2,b2 = classification[idx]
    # print(torch.allclose(a[4:-4],a2[7:]))
    # print(torch.allclose(b[0],b2[0]))
    # print(torch.allclose(b[1],b2[1]))
    # print('')
    #if any of those are false, then print error
    if not torch.allclose(a[4:-4],a2[7:]):
        print('no')
    if not torch.allclose(b[0],b2[0]):
        print('no')
    if not torch.allclose(b[1],b2[1]):
        print('no')
        

In [30]:
print(idx)
print(b)
print(b2)
print(a[4:-4])
print(a2[7:])

tensor([13152039])
(tensor([1]), tensor([0.5150]))
(tensor([1]), tensor([0.5150]))
tensor([ 7,  9,  9,  ..., 10,  9, 10])
tensor([ 7,  9,  9,  ..., 10,  9, 10])


In [33]:
#i'm going to assume that we are probably fine, every value works, let's do one final test
random_ccre = torch.randint(0,14572,(1,))
print(random_ccre)
a,b = classification_multitask[random_ccre]
print(a)
print(b[0],b[1])

tensor([5615])
tensor([ 7, 10,  9,  ..., 10,  9, 10])
tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]) tensor([ 7.9067e-01,  1.1987e-02,  1.2789e+00,  5.1513e-01,  6.7055e-01,
         6.2309e-01,  6.8574e-01,  5.1906e-02,  7.7424e-01, -2.7482e-01,
         1.9750e-01, -2.8510e-02,  3.4374e-01,  1.4384e-01, -3.4549e-01,
         4.4669e-01,  5.7090e-01, -6.6406e-02, -1.5417e-01,  9.8665e-02,
         4.2766e-01, -1.4391e+00,  4.7185e-01,  4.3733e-01,  6.4870e-01,
         2.8997e-01,  8.6265e-01,  7.3861

In [34]:
#only one is closed, find which one is zero
torch.where(b[0] == 0)

(tensor([29]),)

In [36]:
print(b[0][29])
print(b[1][29])
#check to see all values of b[1] that are -10
torch.where(b[1] == -10)

tensor(0)
tensor(-10.)


(tensor([29]),)

In [39]:
#now mask it
mask = b[1] != -10
# print(mask)
print(b[1][mask]==-10)
print(b[1]==-10)

#just the single true, for this case, I think it's safe to say our dataset works properly!!

tensor([False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, 

In [42]:
#we can conclude that the datasets work!
print(isinstance(b[0],torch.Tensor))
print(isinstance(b,tuple))


True
True
